In [1]:
import re
import numpy as np
import pandas as pd
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns
import json
import os
import math
import string

In [2]:
# Libraries for text preprocessing
import nltk
#nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
#nltk.download('wordnet') 
from nltk.stem.wordnet import WordNetLemmatizer

##Creating a list of stop words and adding custom stopwords
stop_words = set(stopwords.words("english"))

In [3]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
np.random.seed(2015)

# Create dataframe that matches paper abstract with subjects

In [4]:
paper_id = []
abstract = []

f = open('aminer_2014.txt','r',encoding = 'utf8')
f.readline()
for i, line in enumerate(f):
        if (i+2) % 250000 == 0:
            print('file '+file+': ',round((i+2)/1000000*100,1),"%")
        json_line = json.loads(line)
        if 'year' in json_line and 'keywords' in json_line and \
        'abstract' in json_line and 'lang' in json_line and \
        'references' in json_line and 'issn' in json_line:
            
            if json_line['lang'] == 'en' :
                
                ## store paper info, later use to get the subject of the paper
                paper_id.append(json_line['id'])
                abstract.append(json_line['abstract'])

                
f.close()

In [5]:
df= pd.DataFrame()
df['id'] = paper_id
df['abstract'] = abstract
df.set_index('id')

,abstract
id,
53e997a2b7602d9701f74cf7,The nursing care of a patient following subara...
53e997a6b7602d9701f7c67f,The authors wish to thank G. W. Beakley and F....
53e997aab7602d9701f827a4,\n Almost all problems known to theoretical ec...
53e997aeb7602d9701f8af9c,Pain management in emergency departments (EDs)...
53e997b5b7602d9701f97a9d,Provides an abstract for each of the two keyno...
53e997bab7602d9701fa1ddc,Howard drifted back into consciousness. For a ...
53e997bab7602d9701fa3207,"Last week, Nature painted a pessimistic pictur..."
53e997c6b7602d9701fb6228,In the first article in the series on risk man...
53e997c6b7602d9701fb7afb,This introduction to the special section on Re...


In [6]:
df.head()

,id,abstract
0,53e997a2b7602d9701f74cf7,The nursing care of a patient following subara...
1,53e997a6b7602d9701f7c67f,The authors wish to thank G. W. Beakley and F....
2,53e997aab7602d9701f827a4,\n Almost all problems known to theoretical ec...
3,53e997aeb7602d9701f8af9c,Pain management in emergency departments (EDs)...
4,53e997b5b7602d9701f97a9d,Provides an abstract for each of the two keyno...


In [7]:
subject = pd.read_csv('paper_subject_match_subfield.csv',index_col = 'id')

In [8]:
subject.head()

,subfield,year
id,,
53e99784b7602d9701f3e13e,1306,2011
53e99784b7602d9701f3e4f2,1307,2002
53e9978db7602d9701f4f415,1300,2005
53e99792b7602d9701f56a86,2738,2004
53e99792b7602d9701f5af0e,1502,1993


In [9]:
tm = pd.merge(df, subject, on = ['id'])

In [10]:
tm.isnull().sum()

id          0
abstract    0
subfield    0
year        0
dtype: int64

In [11]:
len(tm)

234253

In [12]:
tm = tm.dropna()

In [13]:
tm = tm.drop(columns = ['id'])

In [14]:
tm['subfield'] = tm['subfield'].apply(np.int64)

In [15]:
len(tm.subfield.unique())

288

In [16]:
tm.head()

,abstract,subfield,year
0,The nursing care of a patient following subara...,2732,2014
1,The authors wish to thank G. W. Beakley and F....,2736,2014
2,\n Almost all problems known to theoretical ec...,3304,2014
3,Pain management in emergency departments (EDs)...,2907,2014
4,Provides an abstract for each of the two keyno...,2730,2014


### Process representative words for subfield and train TF-IDF model

In [17]:
words = pd.read_csv('words_take150_mf10000.csv')
words.tail()

,0,1,2,3,4,5,6,7,8,9,sub
283,read,refract,visual,eye,chart,ocular,acuiti,intraocular,patient,accommod,3610
284,pharmaci,influenza,student,communiti,pharmacist,vaccin,label,experi,year,immun,3611
285,physic,pain,activ,exercis,particip,sport,athlet,muscl,group,intervent,3612
286,ultrasound,fetal,imag,fetus,patient,sonograph,measur,volum,gestat,case,3614
287,hear,speech,children,languag,nois,swallow,listen,particip,cochlear,measur,3616


In [18]:
len(words)

288

In [19]:
word_list = list()
for i in range(288):
    word_list.append(words.iloc[i,:-1].tolist())

In [20]:
word_list[287]

['hear',
 'speech',
 'children',
 'languag',
 'nois',
 'swallow',
 'listen',
 'particip',
 'cochlear',
 'measur']

In [21]:
#word_list = words['words'].tolist()
#rep_words = [t.translate(str.maketrans('', '', string.punctuation)).split(" ") for t in word_list ]

In [22]:
#print(min([len(ls) for ls in rep_words]))
#print(max([len(ls) for ls in rep_words]))

In [23]:
#import sys
#!{sys.executable} -m pip install gensim
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2015)

In [24]:
from gensim import corpora, models
def LDA_TF_word(doc):
    dictionary = gensim.corpora.Dictionary(doc) 
    bow_corpus = [dictionary.doc2bow(d) for d in doc]
    tfidf = models.TfidfModel(bow_corpus)
    corpus_tfidf = tfidf[bow_corpus]
    lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=20, id2word=dictionary, passes=2, workers=4,minimum_probability=0)
    return(lda_model_tfidf)

In [25]:
rep_model = LDA_TF_word(word_list)

In [26]:
for idx, topic in rep_model.print_topics(-1):
        print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.009*"intervent" + 0.008*"social" + 0.008*"concentr" + 0.008*"physic" + 0.007*"contamin" + 0.006*"metal" + 0.006*"frequenc" + 0.006*"health" + 0.006*"pollut" + 0.005*"particip"
Topic: 1 Word: 0.007*"cortex" + 0.006*"mirna" + 0.006*"brain" + 0.005*"concret" + 0.005*"gill" + 0.005*"shrimp" + 0.004*"matter" + 0.004*"synapt" + 0.004*"wall" + 0.004*"hernia"
Topic: 2 Word: 0.007*"mutat" + 0.006*"sequenc" + 0.006*"genom" + 0.004*"pain" + 0.004*"patient" + 0.004*"speci" + 0.004*"express" + 0.004*"protein" + 0.004*"forens" + 0.004*"histolog"
Topic: 3 Word: 0.009*"children" + 0.007*"electron" + 0.007*"pediatr" + 0.007*"complex" + 0.007*"infant" + 0.006*"ligand" + 0.006*"parent" + 0.006*"reaction" + 0.006*"adolesc" + 0.006*"metal"
Topic: 4 Word: 0.006*"particl" + 0.006*"microgli" + 0.006*"power" + 0.006*"energi" + 0.005*"propos" + 0.005*"nod" + 0.005*"localis" + 0.005*"covari" + 0.005*"test" + 0.005*"imput"
Topic: 5 Word: 0.006*"coat" + 0.005*"genet" + 0.005*"resist" + 0.005*"mate

# Feed Topic model for each subfield

In [106]:
df = tm.groupby('subfield')['abstract'].apply(list).to_dict()

In [107]:
len(df.keys())

288

In [29]:
print(min([len(ls) for ls in df.values()]))
print(max([len(ls) for ls in df.values()]))

1
9217


In [30]:
length = list()
for x in df.values():
    length.append(len(x))

In [31]:
#length.sort()
#length

In [32]:
for key in [key for key in df if len(df[key]) < 30]: del df[key] 

In [33]:
#length1 = list()
#for x in df.values():
#    length1.append(len(x))
#length1.sort()
#length1

In [34]:
len(df)

199

**Processing the abstract:**
- Tokenization: Split the text into sentences and the sentences into words. Lowercase the words and remove punctuation
- Words that have fewer than 3 characters are removed
- All stopwords are removed
- lemmatized — words in third person to first person, verbs in past and future tenses to present
- Stemmed — words are reduced to their root form

In [35]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /Users/yihuan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [36]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2015)
stemmer = SnowballStemmer('english')

In [37]:
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v')) #lemmatize as verb, default is noun

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [38]:
processed_docs = {}
for i in df.keys():
    for j in range(len(df[i])):
        processed_docs.setdefault(i, []).append(preprocess(df[i][j]))

In [39]:
sample_bysub = {}
import random
random.seed(2014)
for i in processed_docs.keys():
    ls = random.sample(range(len(processed_docs[i])), 30)
    for x in ls:
        sample_bysub.setdefault(i, []).append(processed_docs[i][x])

sample 100 papers in arts and feed into each model

In [40]:
dictionary_rep = gensim.corpora.Dictionary(word_list)

In [41]:
def feed_models(text_str):
        bow_vector = dictionary_rep.doc2bow(text_str)
        sc = rep_model[bow_vector]
        score = list()
        for x in range(len(sc)):
            score.append(sc[x][1])
        return score

In [42]:
store_score = {}
for i in sample_bysub.keys():
    for j in range(len(sample_bysub[i])):
        score = feed_models(sample_bysub[i][j])
        store_score.setdefault(i, []).append(score)

In [43]:
store_score[1000]

[[0.006253714,
  0.006253714,
  0.006253714,
  0.006253714,
  0.006253714,
  0.006253714,
  0.006253714,
  0.13115528,
  0.33957765,
  0.006253714,
  0.006253714,
  0.0062537147,
  0.006253714,
  0.006253714,
  0.13120432,
  0.006253714,
  0.006253714,
  0.29800332,
  0.006253714,
  0.006253714],
 [0.0031302175,
  0.0031302175,
  0.1958205,
  0.0031302175,
  0.0031302175,
  0.06872723,
  0.06645802,
  0.0031302175,
  0.0031302175,
  0.0031302175,
  0.0031302178,
  0.118858956,
  0.21979314,
  0.0031302175,
  0.13847408,
  0.0031302175,
  0.067565136,
  0.086740315,
  0.0031302175,
  0.0031302175],
 [0.028518673,
  0.0007157025,
  0.30345276,
  0.060149472,
  0.02920657,
  0.091416575,
  0.02986681,
  0.0007157025,
  0.2591111,
  0.0007157025,
  0.0007157025,
  0.0007157025,
  0.04353839,
  0.0007157025,
  0.0007157025,
  0.04823406,
  0.017702775,
  0.0007157025,
  0.06252598,
  0.020551179],
 [0.103761755,
  0.0009103888,
  0.099599406,
  0.054011963,
  0.0009103888,
  0.0009103888,
 

In [44]:
import numpy as np
from scipy.optimize import minimize
from scipy.spatial.distance import cdist


def geometric_median(points, method='auto', options={}):
    """
    Calculates the geometric median of an array of points.
    method specifies which algorithm to use:
        * 'auto' -- uses a heuristic to pick an algorithm
        * 'minimize' -- scipy.optimize the sum of distances
        * 'weiszfeld' -- Weiszfeld's algorithm
    """

    points = np.asarray(points)

    if len(points.shape) == 1:
        # geometric_median((0, 0)) has too much potential for error.
        # Did the user intend a single 2D point or two scalars?
        # Use np.median if you meant the latter.
        raise ValueError("Expected 2D array")

    if method == 'auto':
        if points.shape[1] > 2:
            # weiszfeld tends to converge faster in higher dimensions
            method = 'weiszfeld'
        else:
            method = 'minimize'

    return _methods[method](points, options)


def minimize_method(points, options={}):
    """
    Geometric median as a convex optimization problem.
    """

    # objective function
    def aggregate_distance(x):
        return cdist([x], points).sum()

    # initial guess: centroid
    centroid = points.mean(axis=0)

    optimize_result = minimize(aggregate_distance, centroid, method='COBYLA')

    return optimize_result.x


def weiszfeld_method(points, options={}):
    """
    Weiszfeld's algorithm as described on Wikipedia.
    """

    default_options = {'maxiter': 1000, 'tol': 1e-7}
    default_options.update(options)
    options = default_options

    def distance_func(x):
        return cdist([x], points)

    # initial guess: centroid
    guess = points.mean(axis=0)

    iters = 0

    while iters < options['maxiter']:
        distances = distance_func(guess).T

        # catch divide by zero
        # TODO: Wikipedia cites how to deal with distance 0
        distances = np.where(distances == 0, 1, distances)

        guess_next = (points/distances).sum(axis=0) / (1./distances).sum(axis=0)

        guess_movement = np.sqrt(((guess - guess_next)**2).sum())

        guess = guess_next

        if guess_movement <= options['tol']:
            break

        iters += 1

    return guess


_methods = {
    'minimize': minimize_method,
    'weiszfeld': weiszfeld_method,
}

In [45]:
median_score = {}
for i in store_score.keys():
    median = geometric_median(store_score[i])
    median_score.setdefault(i, []).append(median)

In [61]:
from scipy.spatial import distance
all_dist = {}
for i in median_score.keys():
    sub_dist = {}
    for j in median_score.keys():
        if i != j:
            dist = distance.euclidean(median_score[i], median_score[j])
            sub_dist.setdefault(j, []).append(dist)
    all_dist.setdefault(i, []).append(sub_dist)

In [62]:
dist_store = {}
for i in median_score.keys():
    sub_dist = {}
    for j in median_score.keys():
        dist = distance.euclidean(median_score[i], median_score[j])
        sub_dist.update({j:dist})
    dist_store.update({i:sub_dist})

In [72]:
df = pd.DataFrame(dist_store)

In [92]:
df1 = pd.DataFrame()
df1["sub1"] = np.repeat(df.index.values,199)

In [100]:
df.index.values.tolist()*2

[1000,
 1100,
 1102,
 1103,
 1104,
 1105,
 1106,
 1108,
 1109,
 1110,
 1201,
 1300,
 1301,
 1302,
 1303,
 1304,
 1305,
 1306,
 1307,
 1308,
 1309,
 1310,
 1311,
 1312,
 1313,
 1314,
 1315,
 1500,
 1502,
 1503,
 1505,
 1600,
 1602,
 1603,
 1604,
 1605,
 1606,
 1607,
 1700,
 1702,
 1705,
 1706,
 1707,
 1710,
 1711,
 1712,
 1900,
 1902,
 1909,
 1910,
 1912,
 2002,
 2103,
 2200,
 2203,
 2204,
 2207,
 2208,
 2209,
 2210,
 2211,
 2213,
 2300,
 2303,
 2304,
 2305,
 2306,
 2307,
 2308,
 2310,
 2312,
 2400,
 2401,
 2402,
 2403,
 2404,
 2405,
 2406,
 2500,
 2502,
 2504,
 2505,
 2506,
 2507,
 2508,
 2600,
 2603,
 2604,
 2605,
 2610,
 2611,
 2613,
 2700,
 2701,
 2702,
 2703,
 2704,
 2705,
 2706,
 2707,
 2708,
 2710,
 2711,
 2712,
 2713,
 2714,
 2715,
 2716,
 2717,
 2718,
 2719,
 2720,
 2721,
 2722,
 2723,
 2724,
 2725,
 2726,
 2727,
 2728,
 2729,
 2730,
 2731,
 2732,
 2733,
 2734,
 2735,
 2736,
 2737,
 2738,
 2739,
 2740,
 2741,
 2742,
 2743,
 2745,
 2746,
 2747,
 2748,
 2800,
 2802,
 2803,
 2804,

In [47]:
import operator
sorted_dist = {}
for i in all_dist.keys():
    st = sorted(all_dist[i][0].items(), key=operator.itemgetter(1))
    sorted_dist.setdefault(i, []).append(st)

In [48]:
sorted_dist[2002] #Economics

[[(2610, [0.0868789238448336]),
  (2600, [0.11049300184655868]),
  (2200, [0.11309083787508784]),
  (3314, [0.11663501552770535]),
  (3312, [0.14118832815968366]),
  (2300, [0.14616774708632244]),
  (1700, [0.15872645417359493]),
  (1606, [0.16243885902619254]),
  (3305, [0.1634210869060381]),
  (2204, [0.16555738032383333]),
  (1710, [0.16636614377351122]),
  (2719, [0.1689782077172446]),
  (1201, [0.169551363241568]),
  (2312, [0.1704682524922136]),
  (2605, [0.17179962280585398]),
  (1100, [0.17233738930842316]),
  (3307, [0.17367593970995485]),
  (2718, [0.17391690179163186]),
  (1900, [0.17602734155989824]),
  (2805, [0.1800416443840588]),
  (2308, [0.18016922320832723]),
  (1109, [0.18025623768059384]),
  (2306, [0.18115795482426347]),
  (3306, [0.18270638802011152]),
  (2611, [0.18331853772029266]),
  (3200, [0.18342547085640615]),
  (2732, [0.18352385396226217]),
  (3206, [0.18426886176473023]),
  (3301, [0.18561486326073878]),
  (3102, [0.18702223716218996]),
  (3104, [0.18766

In [49]:
sorted_dist[1201] #Art

[[(2711, [0.12090537806039205]),
  (3102, [0.123036318587458]),
  (3200, [0.12453654622927472]),
  (2741, [0.12625248948894363]),
  (2805, [0.12793203170375525]),
  (2737, [0.13112824613134028]),
  (2720, [0.13139308001596905]),
  (1310, [0.13226216791698384]),
  (3206, [0.13247298037396743]),
  (2732, [0.1367745195497588]),
  (3612, [0.13727619070192537]),
  (2733, [0.13827589708508226]),
  (2312, [0.13929427939230699]),
  (2727, [0.1403737470186314]),
  (2746, [0.14062949048542522]),
  (1308, [0.14117631275518852]),
  (2809, [0.14131762553807536]),
  (2728, [0.14290293015919664]),
  (2204, [0.1434635819535282]),
  (1314, [0.14443336670643206]),
  (3614, [0.1450053407014851]),
  (2705, [0.1460829558091477]),
  (3205, [0.1481628845710314]),
  (2802, [0.14974896269511404]),
  (2807, [0.14989006947014963]),
  (2800, [0.15092657954326993]),
  (1910, [0.15433892807189953]),
  (3108, [0.15519232456314164]),
  (2808, [0.15524948171601394]),
  (3616, [0.15534983047588252]),
  (2803, [0.155486